In [240]:
import numpy as np
import torch
import random

In [241]:
def score_board(state,a_x,a_y):
    
    print(int(state[7]), int(state[0]), int(state[1]))
    print(int(state[6]), int(world[a_x][a_y]), int(state[2]))
    print(int(state[5]), int(state[4]), int(state[3]))

    print("\nPos :",a_x, a_y)

def gen_world(length,t_x,t_y,a_x,a_y):
    choices = [1,1,0,0,0]

    world = np.random.choice(choices, size=(length,length))

    #agent position
    #target position

    world[t_x][t_y] = -1
    world[a_x][a_y] = 0

    return world

def env(action,state):
    new_state = state.clone().detach()
    
    
    a_x = int(new_state[8]*length -1)
    a_y = int(new_state[9]*length -1)
    # print(a_x,a_y)
    reward = torch.tensor(-1)
    
    if action == "u":
        if a_x == 0:
            reward -= 4
            return reward, new_state, a_x,a_y
        a_x += -1

    elif action == "d":
        if a_x == length-1:
            reward -= 4
            return reward, new_state, a_x,a_y
        a_x += 1
    
    elif action == "r":
        if a_y == length-1:
            reward -= 4
            return reward, new_state, a_x,a_y
        a_y += 1
    
    elif action == "l":
        if a_y == 0:
            reward -= 4
            return reward, new_state, a_x,a_y
        a_y += -1
    
    new_state[8] = (a_x+1)/length
    new_state[9] = (a_y+1)/length
    
    if a_x>0:
        new_state[0] = world[a_x-1][a_y]
    else:
        new_state[0] = 1

    if a_x>0 and a_y<length-1:
        new_state[1] = world[a_x-1][a_y+1]
    else:
        new_state[1] = 1

    if a_y<length-1:
        new_state[2] = world[a_x][a_y+1]
    else:
        new_state[2] = 1
    
    if a_x<length-1 and a_y<length-1:
        new_state[3] = world[a_x+1][a_y+1]
    else:
        new_state[3] = 1
    
    if a_x<length-1:
        new_state[4] = world[a_x+1][a_y]
    else:
        new_state[4] = 1

    if a_x<length-1 and a_y>0:
        new_state[5] = world[a_x+1][a_y-1]
    else:
        new_state[5] = 1

    if a_y>0:    
        new_state[6] = world[a_x][a_y-1]
    else:
        new_state[6] = 1

    if a_x>0 and a_y>0:
        new_state[7] = world[a_x-1][a_y-1]
    else:
        new_state[7] = 1


    if world[a_x][a_y] == 1:
        reward -= 4

    elif world[a_x][a_y]== -1:
        reward += 10
    
    return reward, new_state,a_x,a_y

In [242]:
class FApproxModel(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.linear1 = torch.nn.Linear(10, 20)
        self.linear2 = torch.nn.Linear(20, 10)
        self.linear3 = torch.nn.Linear(10, 4)
    
    def forward(self,features):
        
        out = self.linear1(features)
        out = self.linear2(out)        
        action_pref = self.linear3(out)
        
        return action_pref

def e_greedy_policy(h_f, actions=("u","r","d","l")):
    e = 0.68
    greedy = random.choices((0,1),(1-e,e))[0]
    
    if greedy:
        choice = torch.where(h_f==h_f.max())[0][0]
    else:
        choice = random.choice((0,1,2,3))

    return actions[choice],h_f[choice],0

In [243]:
Q = FApproxModel()

Q.load_state_dict(torch.load("-73|68.pth"))

<All keys matched successfully>

In [244]:
length = 20
a_x, a_y = 0, 0
t_x, t_y = np.random.randint(length/2+6,length), np.random.randint(length/2+6,length)
world = torch.tensor(gen_world(20, t_x,t_y, a_x,a_y),dtype=torch.int8)

In [245]:
def run():
    a_x , a_y = 0, 0
    G = 0
    t = 0

    state = torch.Tensor([1,1,world[0][1],world[1][1],world[1][0],1,1,1,(a_x+1)/length,(a_y+1)/length])
    path_ai = torch.zeros((length,length),dtype=torch.int8)
    path_ai[a_x][a_y] = 1

    while world[a_x][a_y] !=-1 and t<500:
        t+=1
        
        h_f = Q(state)
        a,q_t,p_t = e_greedy_policy(h_f)
        reward, state,a_x,a_y = env(a, state)
        if path_ai[a_x][a_y] + 1 <10:
            path_ai[a_x][a_y] += 1
        G += reward

    if G > -101:
        return G, t, path_ai
    else:
        return run()

G_ai, t_ai, path_ai = run()
print(float(G_ai/t_ai))

-1.399999976158142


In [246]:
a_x, a_y = 0, 0
G_hu = 0
t_hu = 0
reward = 0

state = torch.Tensor([1,1,world[0][1],world[1][1],world[1][0],1,1,1,(a_x+1)/length,(a_y+1)/length])
path_hu = torch.zeros((length,length),dtype=torch.int8)
path_hu[a_x][a_y] = 1

In [247]:
reward,state, a_x,a_y = env("u",state)
# reward,state, a_x,a_y = env("r",state)
# reward,state, a_x,a_y = env("d",state)
# reward,state, a_x,a_y = env("l",state)

t_hu+=1
G_hu+= reward

score_board(state,a_x,a_y)

if path_hu[a_x][a_y] + 1 <10:
    path_hu[a_x][a_y] += 1

print(int(G_hu))

1 1 1
1 0 1
1 1 1

Pos : 0 0
-5


In [248]:
def result():
    print("                   Map")
    for line in world:
        print("                   ",end="")
        for word in line:
            if word != -1:
                print(int(word),end=" ")
            else:
                print("X",end=" ")
        print("")
        
    print("\nAI                                          Human")
    for i in range(length):
        for j in range(length):
                print(int(path_ai[i][j]),end=" ")
        print("    ",end="")
        for j in range(length):
                print(int(path_hu[i][j]),end=" ")
        print("")

    print("Score", int(G_ai), "in", t_ai, "                            Score", int(G_hu),"in",t_hu)

result()


                   Map
                   0 1 0 0 0 1 1 0 0 1 0 1 0 0 1 1 1 1 0 1 
                   1 1 0 0 1 1 1 0 1 0 0 0 0 0 0 0 0 0 0 0 
                   0 0 1 1 0 1 0 0 0 1 0 0 1 1 1 1 1 0 1 0 
                   0 0 1 1 0 0 0 0 1 1 1 0 0 1 1 0 1 0 0 0 
                   1 0 0 0 1 1 0 0 1 1 0 1 0 0 0 0 0 1 1 0 
                   0 0 0 0 1 1 0 0 0 1 0 0 0 0 0 1 0 1 1 0 
                   1 1 1 1 0 0 1 0 0 1 0 1 0 1 0 1 1 0 1 0 
                   0 1 1 0 1 1 0 0 1 0 1 1 0 0 0 0 1 0 1 1 
                   0 0 0 1 1 0 1 1 1 0 0 1 1 0 1 1 0 0 1 1 
                   0 1 1 0 0 1 0 0 1 0 0 1 1 1 1 1 0 0 0 0 
                   1 0 0 1 0 0 0 0 0 1 0 1 0 0 0 0 0 0 1 1 
                   1 0 0 1 0 1 0 1 0 0 0 1 0 1 0 0 1 1 1 0 
                   0 1 0 0 0 1 0 0 1 0 1 1 1 0 1 0 1 0 0 0 
                   0 0 0 0 1 1 1 1 0 0 1 1 1 0 1 1 0 0 1 0 
                   1 0 0 1 0 1 0 0 1 0 0 0 1 0 0 0 1 0 1 1 
                   1 0 0 0 1 0 0 1 0 0 0 1 0 1 1 1 0 0 0 0 
                 